In [ ]:
import pandas as pd

from llm.factory import LLMInterface
from opt.evaluator import evaluate_issue

critic_clients = {
    "qwen3-critic": LLMInterface("openai_like", "qwen3-235b-a22b"),
}

df = pd.read_pickle("test_data.pkl")
# Fix KeyError: 'index' by using df.loc with integer position if 'index' is not a column
zero_confidence_rows = df.loc[[75, 76]]

res = evaluate_issue(critic_clients, zero_confidence_rows)
res


In [ ]:
import os
from sqlalchemy import text

from llm.factory import LLMInterface
from setting.db import db_manager
from utils.json_utils import robust_json_parse
from optimization import improve_graph

llm_client = LLMInterface("openai_like", "qwen3-32b")

session_factory = db_manager.get_session_factory(os.getenv("GRAPH_DATABASE_URI"))

while True:
    try:
        with session_factory() as session:
            sql = text(
                f"""SELECT id, name, description, attributes from entities where updated_at <= '2025-06-21 14:28:44' order by updated_at limit 10"""
            )
            res = session.execute(sql)
            if res.rowcount == 0:
                print("No entities found, complete one round")
                break
            
            entities = {}
            for row in res.fetchall():
                entities[row.id] = {
                    "id": row.id,
                    "name": row.name,
                    "description": row.description,
                    "attributes": row.attributes,
                }
    except Exception as e:
        print(f"Error to get entities: {e}")
        continue

    prompt = f"""Please construct 3-5 natural language queries based on provided entities, these queries will used to similar retrieve (vector similarity) related entities from the graph.

Entities:
{entities}

response in json array format (surround with ```json and ```):
```json
[query1, query2, query3]
```

Now please construct the natural language queries:
"""

    try:
        response = llm_client.generate(prompt)
        queries = robust_json_parse(response, llm_client, expected_format="array")
        print(queries)
    except Exception as e:
        print(f"Error to generate queries: {e}")
        continue

    for query in queries:
        print(f"Improve graph with query: {query}")
        try:
            improve_graph("What are Apple concerning about?")
        except Exception as e:
            print(f"Error to improve graph: {e}")
            continue


In [ ]:
from optimization import improve_graph

improve_graph("What are Apple concerning about?")